In [1]:
# Function to detect keypoints and describe them as feature vectors

In [2]:
pip install opencv-python opencv-contrib-python

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
import cv2
import numpy as np

In [4]:
def keypoints_features(gray_image):
    descriptor =cv2.xfeatures2d.SIFT_create()
    (kpts, features) = descriptor.detectAndCompute(gray_image, None)
    keyPts = []
    keyPts = np.float32([kp.pt for kp in kpts])
    return (keyPts, features)

In [5]:
def match_pts(keypts_1, keypts_2, feature_1, feature_2, ratio = 0.75, thresh = 4.0):
    match_method = cv2.DescriptorMatcher_create("BruteForce")
    nearest_neighbor = 2
    raw_matches = match_method.knnMatch(feature_1, feature_2, nearest_neighbor)
    matches = []
    for match in raw_matches:
        if len(match) ==2 and match[0].distance < match[1].distance * ratio :
            matches.append((match[0].trainIdx , match[0].queryIdx))
    #print(matches)
            
    if len(matches) > 4:
        pts_image1 = []
        pts_image2 = []
        for _, index in matches:
            pts = np.float32(keypts_1[index])
            pts = keypts_1[index]
            pts_image1.append(pts)
        #print("pts_image1", pts_image1)
            
        for index, _ in matches:
            pts = np.float32(keypts_2[index])
            pts = keypts_2[index]
            pts_image2.append(pts)
            
        pts_image1 = np.array(pts_image1, dtype = np.float32)
        pts_image2 = np.array(pts_image2, dtype = np.float32)  
        
        (Homography_matrix, status) = cv2.findHomography(pts_image1, pts_image2, cv2.RANSAC, thresh)
        return (matches, Homography_matrix, status)
    return None
    

In [6]:
def visualize_matches(image1, image2, keyPts_1, keyPts_2, matches, status):
    (h_1, w_1) = image1.shape[0], image1.shape[1]
    (h_2, w_2) = image2.shape[0], image2.shape[1]
    stacked_image = np.zeros((max(h_1, h_2), w_1+w_2 ,3), dtype = 'uint8')
    stacked_image[:h_1 , 0:w_1] = image1
    stacked_image[:h_2 , w_1:] = image2
    
    for ((trainIdx, queryIdx), stat) in zip(matches, status):
        if stat ==1:
            pt_1 = (int(keyPts_1[queryIdx][0]), int(keyPts_1[queryIdx][1]))
            pt_2 = (int(keyPts_2[trainIdx][0])+ w_1, int(keyPts_2[trainIdx][1]))
            cv2.line(stacked_image, pt_1, pt_2, (0,0,255), 1)
    return stacked_image
                                                                 
    

In [7]:
import cv2
image1 = cv2.imread("image1.png")
image2 = cv2.imread("image2.png")

r_w = 400
r_h = 400

image1 = cv2.resize(image1, (r_w, r_h), interpolation = cv2.INTER_AREA)
image2 = cv2.resize(image2, (r_w, r_h), interpolation = cv2.INTER_AREA)

gray_image1 = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)
gray_image2 = cv2.cvtColor(image2, cv2.COLOR_BGR2GRAY)

(kpts_1, features_1) = keypoints_features(gray_image2)
(kpts_2, features_2) = keypoints_features(gray_image1)

M = match_pts (kpts_1, kpts_2, features_1, features_2)
if M is None:
    print("No match found")
    exit()
    
(matches, Homography_matrix, status) = M
stitched_image = cv2.warpPerspective(image2, Homography_matrix, (image1.shape[1] + image2.shape[1], image2.shape[0]))
cv2.imwrite("stitched_image_pre.png", stitched_image)

stitched_image[0: image1.shape[0], 0:image1.shape[1]] = image1
cv2.imwrite("stitched_image.png", stitched_image)

keypts_matchimage = visualize_matches(image1, image2, kpts_1, kpts_2, matches, status)
#cv2.imshow("keypoints match ", keypts_matchimage)
#cv2.imshow("stitched_image", stitched_image)
#cv2.imshow("Input_images and stitched image", np.hstack((image1, image2, stitched_image)))
#cv2.waitKey(0)

# Cropping black region
stitched_image_cropped = stitched_image[0:400, 0:700]
cv2.imwrite("image_stitched_cropped.png", stitched_image_cropped)

print("Size of Stitched_image : ", stitched_image.shape)
print("Size of cropped_image : ", stitched_image_cropped.shape)

                                     


[ WARN:0@0.107] global shadow_sift.hpp:13 SIFT_create DEPRECATED: cv.xfeatures2d.SIFT_create() is deprecated due SIFT tranfer to the main repository. https://github.com/opencv/opencv/issues/16736


Size of Stitched_image :  (400, 800, 3)
Size of cropped_image :  (400, 700, 3)
